In [113]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn import metrics
import matplotlib.pyplot as plt


In [ ]:
Mart_data = pd.read_csv('/content/sample_data/Mart_Train.csv')
Mart_data.head(10)

In [ ]:
Mart_data.isnull().sum()

In [ ]:
Mart_data.info()

**NON-numerical value**

---



Item_Identifier

Item_Fat_Content

Item_Type

Outlet_Identifier

Outlet_Size  (2410 missing values)

Outlet_Location_Type 

Outlet_Type   

In [ ]:
Mart_data['Item_Weight'].fillna(Mart_data['Item_Weight'].mean(),inplace=True)
Mart_data.isnull().sum()

In [ ]:
mode_of_outlet_size = Mart_data.pivot_table(values='Outlet_Size', columns='Outlet_Type',aggfunc=(lambda x: x.mode()[0]))
print(mode_of_outlet_size)


In [ ]:
missing = Mart_data['Outlet_Size'].isnull()
print(missing)

In [ ]:
Mart_data.loc[missing,'Outlet_Size'] = Mart_data.loc[missing,'Outlet_Type'].apply(lambda x: mode_of_outlet_size[x])
Mart_data.head(10)

In [ ]:
sns.set()
Mart_data.describe()

In [ ]:
sns.distplot(Mart_data['Item_MRP'])
plt.figure()
sns.distplot(Mart_data['Item_Weight'])
plt.figure()
sns.distplot(Mart_data['Item_Outlet_Sales'])
plt.figure()
sns.distplot(Mart_data['Item_Visibility'])
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x='Outlet_Type',data=Mart_data)
plt.figure()
sns.countplot(x='Outlet_Size',data=Mart_data)
plt.figure()
sns.countplot(x='Item_Fat_Content',data = Mart_data)
plt.show()

In [153]:
Mart_data['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [155]:
Mart_data.replace({'Item_Fat_Content' : {'LF':'Low Fat','low fat':'Low Fat','reg': 'Regular'}},inplace=True)
Mart_data['Item_Fat_Content'].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

In [ ]:
encoder = LabelEncoder()

Mart_data['Item_Fat_Content'] = encoder.fit_transform(Mart_data['Item_Fat_Content'])
Mart_data['Item_Identifier'] = encoder.fit_transform(Mart_data['Item_Identifier'])
Mart_data['Item_Type'] = encoder.fit_transform(Mart_data['Item_Type'])
Mart_data['Outlet_Identifier'] = encoder.fit_transform(Mart_data['Outlet_Identifier'])
Mart_data['Outlet_Location_Type'] = encoder.fit_transform(Mart_data['Outlet_Location_Type'])
Mart_data['Outlet_Type'] = encoder.fit_transform(Mart_data['Outlet_Type'])
Mart_data['Outlet_Size'] = encoder.fit_transform(Mart_data['Outlet_Size'])

Mart_data.head()


In [160]:
X = Mart_data.drop(columns='Item_Outlet_Sales',axis=1)
Y = Mart_data['Item_Outlet_Sales']

In [161]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size =0.2, random_state=2)

In [162]:
model = XGBRegressor()
model.fit(x_train,y_train)

[15:41:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor()

In [164]:
ypre = model.predict(x_train)
ypre_1 = model.predict(x_test)
R = metrics.r2_score(y_train,ypre)
R_1 = metrics.r2_score(y_test,ypre_1)
print(R, R_1)

0.6364457030941357 0.5867640914432671
